In [1]:
import numpy as np
import matplotlib.pyplot as plt
import log_bessel as lb
from scipy.integrate import quad

# TESTS

In [2]:
v_to_test = np.array([0,3.15e-7, 2.62e-6, 1.3e-5, 9.2e-5, 0.0026,
                      0.0843, 0.17345, 1, 1.63, 7.42, 42.42424, 86.5,
                      113.8, 148.7565, 180.6, 246.3, 300.5, 513.6,
                      712.456, 714.456, 1235.6,8656, 15330.75,
                      37634.2 ,85323])

z_to_test = np.array([1.48e-7, 3.6e-6, 7.248e-5, 4.32e-4, 8.7e-3, 0.04523, 0.17532,
                      1, 3, 11.32465, 105.6, 1038.4, 4236, 11457.6, 62384, 105321.6,
                      158742.3, 196754,  1.98e6])

In [8]:
def logdiffexp(x1,x2):
    
    return x1+np.log1p(-np.exp(-(x1-x2)))

def forward_recursion(v,z):
    
    first = lb.log_bessel_k(v+2,z)[0]
    second = np.log(2) + np.log(v+1) - np.log(z) + lb.log_bessel_k(v+1,z)[0]
    
    return logdiffexp(first,second)

def backward_recursion(v, z):
    
    first = lb.log_bessel_k(v-2, z)[0]
    second = np.log(2)+np.log(v-1)-np.log(z)+lb.log_bessel_k(v-1, z)[0]
    
    return np.logaddexp(first, second)

def recursion_test(v, z):
    
    rhs = np.zeros(np.shape(v))
    
    i_forward = v <= 1
    i_backward = v > 1
    
    rhs[i_forward] = forward_recursion(v[i_forward], z)
    rhs[i_backward] = backward_recursion(v[i_backward], z)
    
    lhs = lb.log_bessel_k(v, z)[0]
    
    return lhs/rhs

In [2]:
v = np.array([0.003, 1.2, 2, 2.8])
z = np.array([0.01, 11, 152])



In [9]:
for z_tmp in z:
    
    
    res = np.zeros(len(v))
    
    i_forward = v <= 1
    i_backward = ~i_forward
    
    first_back = lb.gamma_integral(v[i_backward]-2,z_tmp)
    second_back = np.log(2)+np.log(v[i_backward]-1)-np.log(z_tmp)+lb.gamma_integral(v[i_backward]-1,z_tmp)
    
    first_forw = lb.gamma_integral(v[i_forward]+2,z_tmp)
    second_forw = np.log(2)+np.log(v[i_forward]+1)-np.log(z_tmp)+lb.gamma_integral(v[i_forward]+1,z_tmp)

    rhs_back = np.logaddexp(first_back,second_back)
    lhs_back = lb.gamma_integral(v[i_backward], z_tmp)

    rhs_forw = logdiffexp(first_forw, second_forw)
    lhs_forw = lb.gamma_integral(v[i_forward], z_tmp)
    
    res[i_backward] = lhs_back/rhs_back
    res[i_forward] = lhs_forw/rhs_forw
    
    display(res)

array([0.99999999, 1.        , 1.        , 1.        ])

array([1., 1., 1., 1.])

array([1., 1., 1., 1.])